# 株式分析ツール 利用方法（サンプル）

このノートブックでは、3つの分析パッケージを使った日本株分析のワークフローを学びます。

## 使用するパッケージ

| パッケージ | 主な用途 |
|----------|--------|
| **Market Reader** | J-Quants DBから株価データを取得 |
| **Technical Tools** | テクニカル指標計算・チャート描画 |
| **Stock Screener** | 統合分析結果でのスクリーニング |

## 前提条件
- J-Quants APIでデータ収集済み（`data/jquants.db`が存在）
- 分析スクリプト実行済み（`data/analysis_results.db`が存在）

In [1]:
# 基本インポート
import warnings
from collections import Counter

# 3つのパッケージをインポート
from market_reader import DataReader
from technical_tools import TechnicalAnalyzer, StockScreener, ScreenerFilter

print("インポート完了")

インポート完了


---
## 2. Market Reader: 株価データの取得

`DataReader`はpandas_datareaderライクなインターフェースを提供します。
J-Quants DBに保存された株価データに簡単にアクセスできます。

### 主な機能
- 単一/複数銘柄のデータ取得
- 柔軟なカラム選択（simple/full/カスタム）
- 自動的な日付デフォルト設定
- 4桁/5桁コードの自動正規化

In [2]:
# DataReaderの初期化（デフォルトでsettingsのパスを使用）
reader = DataReader()
print(f"Database path: {reader.db_path}")

Database path: /Users/tak/Markets/Stocks/stock-analysis/data/jquants.db


In [3]:
# 単一銘柄データの取得
# トヨタ(7203)の株価を取得
df_toyota = reader.get_prices("7203", start="2025-01-01", end="2025-12-31")

print(f"データ形状: {df_toyota.shape}")
print(f"インデックス型: {type(df_toyota.index).__name__}")
print(f"カラム: {list(df_toyota.columns)}")
print()
df_toyota.head()

データ形状: (243, 6)
インデックス型: DatetimeIndex
カラム: ['Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentClose']



,Open,High,Low,Close,Volume,AdjustmentClose
Date,,,,,,
2025-01-06,3103.0,3104.0,3000.0,3011.0,41298500.0,3011.0
2025-01-07,3004.0,3127.0,2994.5,3052.0,44700900.0,3052.0
2025-01-08,3050.0,3093.0,3042.0,3073.0,24570000.0,3073.0
2025-01-09,3070.0,3071.0,2997.5,3005.0,26998400.0,3005.0
2025-01-10,2989.0,3027.0,2928.0,2932.0,26369200.0,2932.0


In [4]:
# 複数銘柄の同時取得（MultiIndex DataFrame）
df_multi = reader.get_prices(
    ["7203", "9984", "6758"],  # トヨタ、ソフトバンクG、ソニー
    start="2025-01-01",
    end="2025-03-31",
)

print(f"データ形状: {df_multi.shape}")
print(f"インデックス名: {df_multi.index.names}")
print()

# 特定日のデータを取得
print("=== 2025-01-06のデータ ===")
print(df_multi.loc["2025-01-06"])
print()

# 特定銘柄のみ抽出
print("=== トヨタのみ ===")
toyota_only = df_multi.xs("7203", level="Code")
print(toyota_only.head())

データ形状: (171, 6)
インデックス名: ['Date', 'Code']

=== 2025-01-06のデータ ===
        Open    High     Low   Close      Volume  AdjustmentClose
Code                                                             
6758  3357.0  3358.0  3277.0  3301.0  15195200.0           3301.0
7203  3103.0  3104.0  3000.0  3011.0  41298500.0           3011.0
9984  9299.0  9429.0  9212.0  9234.0   8759300.0           9234.0

=== トヨタのみ ===
              Open    High     Low   Close      Volume  AdjustmentClose
Date                                                                   
2025-01-06  3103.0  3104.0  3000.0  3011.0  41298500.0           3011.0
2025-01-07  3004.0  3127.0  2994.5  3052.0  44700900.0           3052.0
2025-01-08  3050.0  3093.0  3042.0  3073.0  24570000.0           3073.0
2025-01-09  3070.0  3071.0  2997.5  3005.0  26998400.0           3005.0
2025-01-10  2989.0  3027.0  2928.0  2932.0  26369200.0           2932.0


In [5]:
# カラム選択オプション

# columns="simple" (デフォルト): OHLCV + AdjustmentClose
df_simple = reader.get_prices("7203", start="2025-01-01", end="2025-01-10")
print(f"simple: {list(df_simple.columns)}")

# columns="full": 全カラム（株式分割調整済み価格含む）
df_full = reader.get_prices(
    "7203", start="2025-01-01", end="2025-01-10", columns="full"
)
print(f"full: {list(df_full.columns)}")

# カスタム指定
df_custom = reader.get_prices(
    "7203",
    start="2025-01-01",
    end="2025-01-10",
    columns=["AdjustmentOpen", "AdjustmentClose", "AdjustmentVolume"],
)
print(f"custom: {list(df_custom.columns)}")

simple: ['Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentClose']
full: ['Open', 'High', 'Low', 'Close', 'UpperLimit', 'LowerLimit', 'Volume', 'TurnoverValue', 'AdjustmentFactor', 'AdjustmentOpen', 'AdjustmentHigh', 'AdjustmentLow', 'AdjustmentClose', 'AdjustmentVolume']
custom: ['AdjustmentOpen', 'AdjustmentClose', 'AdjustmentVolume']


In [6]:
# 日付のデフォルト動作

# endのみ指定 → startはendから5年前がデフォルト
df_auto_start = reader.get_prices("7203", end="2025-12-31")
print(f"開始日（自動）: {df_auto_start.index.min()}")
print(f"終了日: {df_auto_start.index.max()}")

# 両方省略 → DBの最新日をend、そこから5年前をstart
df_all = reader.get_prices("7203")
print(f"全範囲: {df_all.index.min()} ~ {df_all.index.max()}")
print(f"レコード数: {len(df_all)}")

開始日（自動）: 2021-01-04 00:00:00
終了日: 2025-12-30 00:00:00
全範囲: 2021-02-04 00:00:00 ~ 2026-02-03 00:00:00
レコード数: 1222


In [7]:
# エラーハンドリング
from market_reader import StockNotFoundError

# strict=False（デフォルト）: 存在しない銘柄は警告+空DataFrame
reader_lenient = DataReader(strict=False)
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    df_empty = reader_lenient.get_prices("9999", start="2025-01-01", end="2025-01-31")
    if w:
        print(f"警告: {w[0].message}")
    print(f"空のDataFrame: {df_empty.empty}")

# strict=True: 存在しない銘柄は例外
reader_strict = DataReader(strict=True)
try:
    reader_strict.get_prices("9999", start="2025-01-01", end="2025-01-31")
except StockNotFoundError as e:
    print(f"例外発生: {e}")

警告: No data found for stock code: ['9999']
空のDataFrame: True
例外発生: Stock code not found: 9999


---
## 3. Technical Tools: テクニカル分析

`TechnicalAnalyzer`は日本株（J-Quants）と米国株（yfinance）の両方に対応したテクニカル分析ツールです。

### 主な機能
- 複数のテクニカル指標計算（SMA、EMA、RSI、MACD、ボリンジャーバンド）
- ゴールデンクロス/デッドクロスの自動検出
- Plotlyによるインタラクティブチャート生成
- 既存の分析結果（Minervini、RSP等）との連携

In [8]:
# TechnicalAnalyzerの初期化

# 日本株用（J-Quantsソース）
analyzer_jp = TechnicalAnalyzer(source="jquants")

# 米国株用（yfinanceソース）
analyzer_us = TechnicalAnalyzer(source="yfinance")

print(f"日本株用アナライザー: source={analyzer_jp._source_name}")
print(f"米国株用アナライザー: source={analyzer_us._source_name}")

日本株用アナライザー: source=jquants
米国株用アナライザー: source=yfinance


In [9]:
# 価格データの取得（内部でキャッシュされる）
df_prices = analyzer_jp.get_prices("7203", start="2025-01-01", end="2025-12-31")
print(f"カラム: {list(df_prices.columns)}")
df_prices.tail()

カラム: ['Open', 'High', 'Low', 'Close', 'Volume']


,Open,High,Low,Close,Volume
Date,,,,,
2025-12-24,3418.0,3420.0,3353.0,3353.0,12845800.0
2025-12-25,3377.0,3383.0,3358.0,3374.0,7132300.0
2025-12-26,3377.0,3397.0,3369.0,3380.0,10705500.0
2025-12-29,3386.0,3386.0,3352.0,3364.0,12099700.0
2025-12-30,3370.0,3387.0,3353.0,3356.0,12175900.0


---
## 4. テクニカル指標の追加

各種テクニカル指標をDataFrameに追加できます。

In [10]:
# SMA（単純移動平均）の追加
df_with_sma = analyzer_jp.add_sma("7203", periods=[5, 25, 75], start="2025-01-01")

# SMAカラムを確認
sma_cols = [col for col in df_with_sma.columns if col.startswith("SMA_")]
print(f"追加されたSMAカラム: {sma_cols}")
df_with_sma[["Close"] + sma_cols].tail(10)

追加されたSMAカラム: ['SMA_5', 'SMA_25', 'SMA_75']


,Close,SMA_5,SMA_25,SMA_75
Date,,,,
2026-01-21,3552.0,3620.0,3439.24,3183.220000
2026-01-22,3584.0,3594.0,3452.20,3193.013333
2026-01-23,3624.0,3584.8,3463.16,3203.713333
2026-01-26,3477.0,3555.6,3469.04,3212.673333
2026-01-27,3459.0,3539.2,3473.44,3220.933333
2026-01-28,3347.0,3498.2,3472.80,3225.906667
2026-01-29,3448.0,3471.0,3473.76,3231.573333
2026-01-30,3504.0,3447.0,3475.72,3238.173333
2026-02-02,3535.0,3458.6,3480.52,3246.006667


In [11]:
# EMA（指数移動平均）の追加
df_with_ema = analyzer_jp.add_ema("7203", periods=[12, 26], start="2025-01-01")
ema_cols = [col for col in df_with_ema.columns if col.startswith("EMA_")]
print(f"追加されたEMAカラム: {ema_cols}")
df_with_ema[["Close"] + ema_cols].tail()

追加されたEMAカラム: ['EMA_12', 'EMA_26']


,Close,EMA_12,EMA_26
Date,,,
2026-01-28,3347.0,3499.927138,3449.117816
2026-01-29,3448.0,3491.938347,3449.035014
2026-01-30,3504.0,3493.793986,3453.106495
2026-02-02,3535.0,3500.133373,3459.172680
2026-02-03,3594.0,3514.574392,3469.159889


In [12]:
# RSI（相対力指数）の追加
# デフォルトは14日
df_with_rsi = analyzer_jp.add_rsi("7203", period=14, start="2025-01-01")
df_with_rsi[["Close", "RSI_14"]].tail(10)

,Close,RSI_14
Date,,
2026-01-21,3552.0,58.850939
2026-01-22,3584.0,60.603707
2026-01-23,3624.0,62.740191
2026-01-26,3477.0,51.653831
2026-01-27,3459.0,50.477629
2026-01-28,3347.0,43.795177
2026-01-29,3448.0,50.198003
2026-01-30,3504.0,53.369878
2026-02-02,3535.0,55.075606


In [13]:
# MACD（移動平均収束発散）の追加
# デフォルト: fast=12, slow=26, signal=9
df_with_macd = analyzer_jp.add_macd(
    "7203", fast=12, slow=26, signal=9, start="2025-01-01"
)

macd_cols = ["MACD", "MACD_Signal", "MACD_Hist"]
df_with_macd[["Close"] + macd_cols].tail(10)

,Close,MACD,MACD_Signal,MACD_Hist
Date,,,,
2026-01-21,3552.0,99.592403,94.678333,4.914070
2026-01-22,3584.0,96.496827,95.042032,1.454795
2026-01-23,3624.0,96.162722,95.266170,0.896552
2026-01-26,3477.0,83.078592,92.828654,-9.750062
2026-01-27,3459.0,70.444831,88.351890,-17.907059
2026-01-28,3347.0,50.809322,80.843376,-30.034054
2026-01-29,3448.0,42.903333,73.255367,-30.352035
2026-01-30,3504.0,40.687491,66.741792,-26.054301
2026-02-02,3535.0,40.960692,61.585572,-20.624880


In [14]:
# ボリンジャーバンドの追加
# デフォルト: period=20, std=2.0
df_with_bb = analyzer_jp.add_bollinger_bands(
    "7203", period=20, std=2.0, start="2025-01-01"
)

bb_cols = ["BB_Upper", "BB_Middle", "BB_Lower"]
df_with_bb[["Close"] + bb_cols].tail(10)

,Close,BB_Upper,BB_Middle,BB_Lower
Date,,,,
2026-01-21,3552.0,3721.938366,3466.45,3210.961634
2026-01-22,3584.0,3734.324038,3474.45,3214.575962
2026-01-23,3624.0,3750.972182,3482.90,3214.827818
2026-01-26,3477.0,3752.193440,3486.00,3219.806560
2026-01-27,3459.0,3750.472042,3491.30,3232.127958
2026-01-28,3347.0,3751.960426,3489.95,3227.939574
2026-01-29,3448.0,3751.082848,3493.35,3235.617152
2026-01-30,3504.0,3750.792367,3500.35,3249.907633
2026-02-02,3535.0,3750.650673,3509.30,3267.949327


In [15]:
# 複数指標の一括計算
df_all_indicators = analyzer_jp.calculate_indicators(
    "7203", indicators=["sma", "rsi", "macd", "bb"], start="2025-01-01"
)

print(f"全カラム: {list(df_all_indicators.columns)}")

全カラム: ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_25', 'SMA_75', 'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_Upper', 'BB_Middle', 'BB_Lower']


---
## 5. シグナル検出（ゴールデンクロス/デッドクロス）

移動平均のクロス（交差）を自動検出します。

| シグナル名 | 意味 |
|---------|------|
| Golden Cross | 短期MAが長期MAを上抜け（買いシグナル） |
| Dead Cross | 短期MAが長期MAを下抜け（売りシグナル） |

In [16]:
# 単一パターンのクロス検出（5日線と25日線）
signals = analyzer_jp.detect_crosses("7203", short=5, long=25, start="2025-01-01")

print(f"検出されたシグナル数: {len(signals)}")
for sig in signals[-5:]:  # 最新5件
    print(f"  {sig.date.strftime('%Y-%m-%d')}: {sig.signal_type} @ {sig.price:.2f}")

検出されたシグナル数: 16
  2025-10-16: dead_cross @ 2954.50
  2025-10-20: golden_cross @ 3003.00
  2025-11-20: dead_cross @ 3044.00
  2025-12-12: golden_cross @ 3260.00
  2026-01-29: dead_cross @ 3448.00


In [17]:
# 複数パターンのクロス検出
patterns = [(5, 25), (25, 75)]  # 5/25日線、25/75日線
signals_multi = analyzer_jp.detect_crosses(
    "7203", patterns=patterns, start="2025-01-01"
)

print(f"検出されたシグナル数: {len(signals_multi)}")

# シグナル種別ごとにグループ化
signal_counts = Counter(
    f"{s.signal_type} ({s.short_period}/{s.long_period})" for s in signals_multi
)
for sig_type, count in signal_counts.items():
    print(f"  {sig_type}: {count}件")

検出されたシグナル数: 19
  golden_cross (5/25): 8件
  dead_cross (5/25): 8件
  golden_cross (25/75): 2件
  dead_cross (25/75): 1件


In [18]:
# Signalオブジェクトの構造確認
if signals:
    sig = signals[-1]
    print("Signal attributes:")
    print(f"  date: {sig.date} (type: {type(sig.date).__name__})")
    print(f"  signal_type: {sig.signal_type}")
    print(f"  price: {sig.price}")
    print(f"  short_period: {sig.short_period}")
    print(f"  long_period: {sig.long_period}")

Signal attributes:
  date: 2026-01-29 00:00:00 (type: datetime)
  signal_type: dead_cross
  price: 3448.0
  short_period: 5
  long_period: 25


---
## 6. インタラクティブチャート

`plot_chart()`メソッドでPlotlyによるインタラクティブなローソク足チャートを生成します。

### 表示オプション
- `show_sma`: 移動平均線（リストで期間指定）
- `show_bb`: ボリンジャーバンド
- `show_rsi`: RSIサブプロット
- `show_macd`: MACDサブプロット
- `show_signals`: クロスシグナルマーカー

In [19]:
# 基本チャート（ローソク足のみ）
fig = analyzer_jp.plot_chart("7203", start="2025-01-01")
fig.show()

In [20]:
# 移動平均線付きチャート
fig = analyzer_jp.plot_chart("7203", show_sma=[5, 25, 75], start="2025-01-01")
fig.show()

In [21]:
# フル機能チャート（全指標表示）
fig = analyzer_jp.plot_chart(
    "7203",
    show_sma=[5, 25, 75],
    show_bb=True,
    show_rsi=True,
    show_macd=True,
    show_signals=True,
    signal_patterns=[(5, 25), (25, 75)],
    start="2025-01-01",
)
fig.show()

In [22]:
# 米国株のチャート（yfinance経由）
fig_us = analyzer_us.plot_chart(
    "AAPL",
    show_sma=[50, 200],
    show_rsi=True,
    period="1y",  # yfinance固有のパラメータ
)
fig_us.show()

### 6-1. テクニカル指標別チャートサンプル

各テクニカル指標を個別に表示するサンプルです。Plotlyを使ってカスタムチャートも作成できます。

In [23]:
# ボリンジャーバンド付きチャート
# 価格がバンドの上下どちらに位置しているかを視覚的に確認
fig = analyzer_jp.plot_chart(
    "7203",
    show_bb=True,  # ボリンジャーバンドのみ
    start="2025-01-01",
)
fig.update_layout(title="7203: ボリンジャーバンド（20日, 2σ）")
fig.show()

In [24]:
# RSI付きチャート
# RSI > 70: 買われすぎ、RSI < 30: 売られすぎ
fig = analyzer_jp.plot_chart(
    "7203",
    show_rsi=True,  # RSIサブプロット表示
    start="2025-01-01",
)
fig.update_layout(title="7203: RSI（14日）- 70以上で買われすぎ、30以下で売られすぎ")
fig.show()

In [25]:
# MACD付きチャート
# MACDがシグナル線を上抜け: 買いシグナル、下抜け: 売りシグナル
fig = analyzer_jp.plot_chart(
    "7203",
    show_macd=True,  # MACDサブプロット表示
    start="2025-01-01",
)
fig.update_layout(title="7203: MACD（12/26/9）")
fig.show()

In [26]:
# クロスシグナル付きチャート
# ゴールデンクロス（緑▲）とデッドクロス（赤▼）を表示
fig = analyzer_jp.plot_chart(
    "7203",
    show_sma=[5, 25],
    show_signals=True,
    signal_patterns=[(5, 25)],  # 5日線と25日線のクロスを検出
    start="2025-01-01",
)
fig.update_layout(title="7203: SMA(5/25) + クロスシグナル")
fig.show()

In [27]:
# RSI + MACD 同時表示（トレンド確認に有効）
fig = analyzer_jp.plot_chart(
    "7203", show_sma=[25, 75], show_rsi=True, show_macd=True, start="2025-01-01"
)
fig.update_layout(title="7203: RSI + MACD 複合分析")
fig.show()

### 6-1-2. Plotlyでカスタムチャートを作成

`TechnicalAnalyzer`で計算した指標データを使って、Plotlyで独自のチャートを構築することもできます。

In [28]:
# カスタムRSIチャート（買われすぎ/売られすぎゾーンをハイライト）
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# RSI付きデータを取得
df_rsi = analyzer_jp.add_rsi("7203", period=14, start="2025-01-01")

# サブプロット作成（上: 価格、下: RSI）
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3]
)

# ローソク足
fig.add_trace(
    go.Candlestick(
        x=df_rsi.index,
        open=df_rsi["Open"],
        high=df_rsi["High"],
        low=df_rsi["Low"],
        close=df_rsi["Close"],
        name="OHLC",
    ),
    row=1,
    col=1,
)

# RSIライン
fig.add_trace(
    go.Scatter(
        x=df_rsi.index,
        y=df_rsi["RSI_14"],
        name="RSI(14)",
        line=dict(color="purple", width=1.5),
    ),
    row=2,
    col=1,
)

# 買われすぎライン（70）
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
# 売られすぎライン（30）
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
# 中央ライン（50）
fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

# 買われすぎゾーン（70-100）を薄い赤でハイライト
fig.add_hrect(y0=70, y1=100, fillcolor="red", opacity=0.1, row=2, col=1)
# 売られすぎゾーン（0-30）を薄い緑でハイライト
fig.add_hrect(y0=0, y1=30, fillcolor="green", opacity=0.1, row=2, col=1)

fig.update_layout(
    title="7203: カスタムRSIチャート（ゾーンハイライト付き）",
    xaxis_rangeslider_visible=False,
    height=600,
)
fig.update_yaxes(title_text="価格", row=1, col=1)
fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1)

fig.show()

In [29]:
# カスタムMACDチャート（ヒストグラムを色分け表示）
# MACD付きデータを取得
df_macd = analyzer_jp.add_macd("7203", fast=12, slow=26, signal=9, start="2025-01-01")

# サブプロット作成（上: 価格、下: MACD）
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3]
)

# ローソク足
fig.add_trace(
    go.Candlestick(
        x=df_macd.index,
        open=df_macd["Open"],
        high=df_macd["High"],
        low=df_macd["Low"],
        close=df_macd["Close"],
        name="OHLC",
    ),
    row=1,
    col=1,
)

# MACDライン
fig.add_trace(
    go.Scatter(
        x=df_macd.index,
        y=df_macd["MACD"],
        name="MACD",
        line=dict(color="blue", width=1.5),
    ),
    row=2,
    col=1,
)

# シグナルライン
fig.add_trace(
    go.Scatter(
        x=df_macd.index,
        y=df_macd["MACD_Signal"],
        name="Signal",
        line=dict(color="orange", width=1.5),
    ),
    row=2,
    col=1,
)

# ヒストグラム（色分け: 正は緑、負は赤）
colors = ["green" if val >= 0 else "red" for val in df_macd["MACD_Hist"]]
fig.add_trace(
    go.Bar(
        x=df_macd.index,
        y=df_macd["MACD_Hist"],
        name="Histogram",
        marker_color=colors,
        opacity=0.5,
    ),
    row=2,
    col=1,
)

# ゼロライン
fig.add_hline(y=0, line_dash="solid", line_color="gray", row=2, col=1)

fig.update_layout(
    title="7203: カスタムMACDチャート（ヒストグラム色分け）",
    xaxis_rangeslider_visible=False,
    height=600,
    barmode="overlay",
)
fig.update_yaxes(title_text="価格", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)

fig.show()

In [30]:
# カスタムボリンジャーバンドチャート（%Bインジケーター付き）
# %B = (終値 - 下限バンド) / (上限バンド - 下限バンド)
# %B > 1: 上限バンドを上抜け、%B < 0: 下限バンドを下抜け

df_bb = analyzer_jp.add_bollinger_bands("7203", period=20, std=2.0, start="2025-01-01")

# %Bを計算
df_bb["BB_PercentB"] = (df_bb["Close"] - df_bb["BB_Lower"]) / (
    df_bb["BB_Upper"] - df_bb["BB_Lower"]
)

# サブプロット作成
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3]
)

# ローソク足
fig.add_trace(
    go.Candlestick(
        x=df_bb.index,
        open=df_bb["Open"],
        high=df_bb["High"],
        low=df_bb["Low"],
        close=df_bb["Close"],
        name="OHLC",
    ),
    row=1,
    col=1,
)

# ボリンジャーバンド（上限）
fig.add_trace(
    go.Scatter(
        x=df_bb.index,
        y=df_bb["BB_Upper"],
        name="Upper Band",
        line=dict(color="rgba(173, 216, 230, 0.8)", width=1),
    ),
    row=1,
    col=1,
)

# ボリンジャーバンド（中央）
fig.add_trace(
    go.Scatter(
        x=df_bb.index,
        y=df_bb["BB_Middle"],
        name="Middle Band",
        line=dict(color="blue", width=1, dash="dot"),
    ),
    row=1,
    col=1,
)

# ボリンジャーバンド（下限）
fig.add_trace(
    go.Scatter(
        x=df_bb.index,
        y=df_bb["BB_Lower"],
        name="Lower Band",
        line=dict(color="rgba(173, 216, 230, 0.8)", width=1),
        fill="tonexty",
        fillcolor="rgba(173, 216, 230, 0.2)",
    ),
    row=1,
    col=1,
)

# %Bインジケーター
fig.add_trace(
    go.Scatter(
        x=df_bb.index,
        y=df_bb["BB_PercentB"],
        name="%B",
        line=dict(color="purple", width=1.5),
    ),
    row=2,
    col=1,
)

# 基準ライン
fig.add_hline(y=1.0, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=0.5, line_dash="dot", line_color="gray", row=2, col=1)
fig.add_hline(y=0.0, line_dash="dash", line_color="green", row=2, col=1)

fig.update_layout(
    title="7203: ボリンジャーバンド + %Bインジケーター",
    xaxis_rangeslider_visible=False,
    height=600,
)
fig.update_yaxes(title_text="価格", row=1, col=1)
fig.update_yaxes(title_text="%B", row=2, col=1)

fig.show()

In [31]:
# 出来高付きチャート（価格と出来高の関係を分析）
df_vol = analyzer_jp.get_prices("7203", start="2025-01-01")

# サブプロット作成（上: 価格、下: 出来高）
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3]
)

# ローソク足
fig.add_trace(
    go.Candlestick(
        x=df_vol.index,
        open=df_vol["Open"],
        high=df_vol["High"],
        low=df_vol["Low"],
        close=df_vol["Close"],
        name="OHLC",
    ),
    row=1,
    col=1,
)

# 出来高（上昇日: 緑、下落日: 赤）
colors = [
    "green" if close >= open_ else "red"
    for close, open_ in zip(df_vol["Close"], df_vol["Open"])
]
fig.add_trace(
    go.Bar(
        x=df_vol.index,
        y=df_vol["Volume"],
        name="Volume",
        marker_color=colors,
        opacity=0.7,
    ),
    row=2,
    col=1,
)

# 出来高の移動平均（20日）
vol_ma = df_vol["Volume"].rolling(window=20).mean()
fig.add_trace(
    go.Scatter(
        x=df_vol.index,
        y=vol_ma,
        name="Vol MA(20)",
        line=dict(color="orange", width=1.5),
    ),
    row=2,
    col=1,
)

fig.update_layout(
    title="7203: 価格 + 出来高チャート", xaxis_rangeslider_visible=False, height=600
)
fig.update_yaxes(title_text="価格", row=1, col=1)
fig.update_yaxes(title_text="出来高", row=2, col=1)

fig.show()

---
## 6-2. 既存分析結果との連携

`load_existing_analysis()`で、日次バッチで計算済みのMinerviniスクリーニング結果やRSP（相対力）データを取得できます。

In [32]:
# 既存の分析結果を取得
existing = analyzer_jp.load_existing_analysis("7203")

print("=== Minervini分析結果 ===")
if existing["minervini"]:
    for key, value in existing["minervini"].items():
        print(f"  {key}: {value}")
else:
    print("  データなし")

print()
print("=== Relative Strength分析結果 ===")
if existing["relative_strength"]:
    for key, value in existing["relative_strength"].items():
        print(f"  {key}: {value}")
else:
    print("  データなし")

=== Minervini分析結果 ===
  Date: 2026-02-03
  Code: 72030
  Close: 3594.0
  Sma50: 3336.78
  Sma150: 3012.713333333333
  Sma200: 2916.5825
  Type_1: 1.0
  Type_2: 1.0
  Type_3: 1.0
  Type_4: 1.0
  Type_5: 1.0
  Type_6: 1.0
  Type_7: 1.0
  Type_8: 1.0

=== Relative Strength分析結果 ===
  Date: 2026-02-03
  Code: 72030
  RelativeStrengthPercentage: 12.659856555671272
  RelativeStrengthIndex: 73.28170354508684


---
## 7. Stock Screener: 銘柄スクリーニング

`StockScreener`は統合分析結果（integrated_scores）をベースに、テクニカル指標・財務指標・チャートパターンで銘柄をフィルタリングします。

### フィルタリング可能な指標

| カテゴリ | 指標 | パラメータ例 |
|---------|-----|-------------|
| テクニカル | 統合スコア | `composite_score_min=70.0` |
| テクニカル | HL比率 | `hl_ratio_min=80.0` |
| テクニカル | RSI | `rsi_max=70.0` |
| 財務 | 時価総額 | `market_cap_min=100_000_000_000` |
| 財務 | PER | `per_max=15.0` |
| 財務 | PBR | `pbr_max=2.0` |
| 財務 | ROE | `roe_min=15.0` |
| 財務 | 配当利回り | `dividend_yield_min=2.0` |
| パターン | チャート分類 | `pattern_window=60, pattern_labels=["上昇"]` |

In [33]:
# StockScreenerの初期化（デフォルトでsettingsのパスを使用）
screener = StockScreener()
print(f"Analysis DB: {screener.analysis_db_path}")
print(f"Statements DB: {screener.statements_db_path}")

Analysis DB: /Users/tak/Markets/Stocks/stock-analysis/data/analysis_results.db
Statements DB: /Users/tak/Markets/Stocks/stock-analysis/data/statements.db


In [34]:
# 基本フィルタリング（パラメータなしで最新日の全データ取得、上位100件）
results = screener.filter()
print(f"取得件数: {len(results)}")
print(f"カラム: {list(results.columns)}")
results.head(10)

取得件数: 100
カラム: ['Date', 'Code', 'composite_score', 'composite_score_rank', 'hl_ratio_rank', 'rsp_rank', 'HlRatio', 'MedianRatio', 'RelativeStrengthPercentage', 'RelativeStrengthIndex']


,Date,Code,composite_score,composite_score_rank,hl_ratio_rank,rsp_rank,HlRatio,MedianRatio,RelativeStrengthPercentage,RelativeStrengthIndex
0,2026-02-03,39020,99.191358,1.0,121.0,7.0,99.118295,9.294636,110.356277,98.860100
1,2026-02-03,40040,99.124340,2.0,1.0,50.0,100.000000,21.452750,45.943831,97.810849
2,2026-02-03,31030,98.988316,3.0,164.0,12.0,98.727273,10.181818,75.292474,98.743517
3,2026-02-03,83930,98.909826,4.0,1.0,71.0,100.000000,24.773078,40.854208,97.274566
4,2026-02-03,83920,98.704604,5.0,100.0,66.0,99.347045,32.337578,41.891180,97.414466
5,2026-02-03,64070,98.695313,6.0,1.0,92.0,100.000000,33.187524,36.438929,96.738282
6,2026-02-03,62270,98.689766,7.0,193.0,28.0,98.377282,22.177147,53.129605,98.347133
7,2026-02-03,83860,98.625690,8.0,80.0,80.0,99.522825,35.859711,38.809456,97.041399
8,2026-02-03,66130,98.564739,9.0,1.0,106.0,100.000000,29.096045,34.708848,96.411849
9,2026-02-03,58030,98.561853,10.0,188.0,43.0,98.430566,26.025544,48.060932,97.974066


In [35]:
# テクニカル指標でのフィルタリング
# 統合スコア70以上、HL比率80以上、RSI70以下（過熱感なし）
results_tech = screener.filter(
    composite_score_min=70.0, hl_ratio_min=80.0, rsi_max=70.0, limit=50
)
print(f"条件に合致: {len(results_tech)}件")
results_tech.head()

条件に合致: 50件


,Date,Code,composite_score,composite_score_rank,hl_ratio_rank,rsp_rank,HlRatio,MedianRatio,RelativeStrengthPercentage,RelativeStrengthIndex
741,2026-02-03,52010,85.115227,742,1,1157,100.000000,37.165510,11.276344,69.038068
745,2026-02-03,40420,85.005343,746,130,1137,99.025788,50.744986,11.484000,69.737568
757,2026-02-03,40450,84.742160,758,1,1193,100.000000,43.583333,11.011860,68.105401
761,2026-02-03,64580,84.635123,762,219,1136,98.076923,49.711538,11.486233,69.760885
770,2026-02-03,80180,84.465954,771,137,1180,98.983051,38.644068,11.105510,68.431834


In [36]:
# 財務指標との組み合わせ（テクニカル + ファンダメンタルズ）
results_combined = screener.filter(
    composite_score_min=65.0,
    market_cap_min=100_000_000_000,  # 時価総額1000億円以上
    per_max=20.0,  # PER 20倍以下
    roe_min=10.0,  # ROE 10%以上
    limit=30,
)
print(f"条件に合致: {len(results_combined)}件")
results_combined.head()

条件に合致: 30件


,Date,Code,composite_score,composite_score_rank,hl_ratio_rank,rsp_rank,HlRatio,MedianRatio,RelativeStrengthPercentage,RelativeStrengthIndex,longName,sector,marketCap,trailingPE,priceToBook,dividendYield,returnOnEquity
25,2026-02-03,77350,98.189020,26,148,96,98.827534,27.817700,35.448330,96.645015,ＳＣＲＥＥＮホールディングス,電気機器,1.983540e+12,19.08,4.44,1.58,23.64
28,2026-02-03,67870,98.127139,29,145,105,98.882682,40.020315,34.779713,96.435165,メイコー,電気機器,2.755381e+11,18.05,2.61,0.86,12.91
31,2026-02-03,80020,98.000469,32,40,158,99.941691,32.981050,30.194080,95.059481,丸紅,卸売業,8.513047e+12,16.93,2.34,1.85,13.35
48,2026-02-03,64900,97.521470,49,114,174,99.210526,32.105263,29.164064,94.593148,ＰＩＬＬＡＲ,機械,1.542612e+11,17.31,1.95,2.03,11.24
94,2026-02-03,67450,96.456971,95,170,256,98.671096,35.714286,24.900717,92.471330,ホーチキ,電気機器,1.294920e+11,15.93,2.06,1.63,12.90


In [37]:
# チャートパターンでのフィルタリング
# 60日窓で「上昇」「急上昇」パターンの銘柄
results_pattern = screener.filter(
    pattern_window=60,
    pattern_labels=["上昇", "急上昇"],
    composite_score_min=60.0,
    limit=30,
)
print(f"上昇トレンド銘柄: {len(results_pattern)}件")
results_pattern.head()

上昇トレンド銘柄: 30件


,Date,Code,composite_score,composite_score_rank,hl_ratio_rank,rsp_rank,HlRatio,MedianRatio,RelativeStrengthPercentage,RelativeStrengthIndex,pattern_label,score
0,2026-02-03,39020,99.191358,1,121,7,99.118295,9.294636,110.356277,98.860100,上昇,0.880025
1,2026-02-03,40040,99.124340,2,1,50,100.000000,21.452750,45.943831,97.810849,急上昇,0.958136
2,2026-02-03,31030,98.988316,3,164,12,98.727273,10.181818,75.292474,98.743517,急上昇,0.855983
3,2026-02-03,83930,98.909826,4,1,71,100.000000,24.773078,40.854208,97.274566,上昇,0.979847
4,2026-02-03,83920,98.704604,5,100,66,99.347045,32.337578,41.891180,97.414466,上昇,0.967575


---
## 8. ScreenerFilterによる構造化パラメータ

`ScreenerFilter`データクラスを使うと、フィルタ条件を再利用可能なオブジェクトとして管理できます。

In [38]:
# ScreenerFilterでパラメータを構造化
config = ScreenerFilter(
    composite_score_min=70.0,
    hl_ratio_min=80.0,
    market_cap_min=50_000_000_000,  # 500億円以上
    per_max=25.0,
    limit=50,
)

# filterメソッドにそのまま渡せる
results_filtered = screener.filter(config)
print(f"条件に合致: {len(results_filtered)}件")

条件に合致: 50件


In [39]:
# フィルタ設定の再利用

# 「成長株」フィルタを定義
growth_filter = ScreenerFilter(
    composite_score_min=75.0, hl_ratio_min=85.0, roe_min=15.0, limit=30
)

# 「バリュー株」フィルタを定義
value_filter = ScreenerFilter(
    composite_score_min=60.0,
    per_max=12.0,
    pbr_max=1.5,
    dividend_yield_min=3.0,
    limit=30,
)

# それぞれのフィルタを適用
print("=== 成長株候補 ===")
growth_stocks = screener.filter(growth_filter)
print(f"{len(growth_stocks)}件")

print()
print("=== バリュー株候補 ===")
value_stocks = screener.filter(value_filter)
print(f"{len(value_stocks)}件")

=== 成長株候補 ===
30件

=== バリュー株候補 ===
30件


In [40]:
# to_dict()による設定内容の確認（Noneは除外される）
print("Growth filter settings:")
for key, value in growth_filter.to_dict().items():
    print(f"  {key}: {value}")

Growth filter settings:
  composite_score_min: 75.0
  hl_ratio_min: 85.0
  roe_min: 15.0
  limit: 30


---
## 9. ランク変動分析

`rank_changes()`メソッドで、指定期間内に順位が大きく変動した銘柄を検出できます。

> **注意**: このメソッドは過去データとの比較を行うため、`integrated_scores`テーブルに**複数日分のデータ**が必要です。  
> 日次分析(`run_daily_analysis.py`)を複数日実行してデータを蓄積した後に利用してください。  
> データが1日分しかない場合、結果は0件になります。

### メトリクス
- `composite_score`: 統合スコアの順位
- `hl_ratio`: HL比率の順位
- `rsp`: 相対力（RSP）の順位

In [41]:
# 7日間で順位が上昇した銘柄
# 注: 複数日分のデータが必要（データが1日分の場合は0件になります）
movers_up = screener.rank_changes(
    metric="composite_score",
    days=7,
    direction="up",
    min_change=10,  # 10位以上の上昇
    limit=20,
)
print(f"順位上昇銘柄: {len(movers_up)}件")
if movers_up.empty:
    print("  → データが1日分しかないか、条件に合う銘柄がありません")
else:
    display(movers_up.head(10))

順位上昇銘柄: 0件
  → データが1日分しかないか、条件に合う銘柄がありません


In [42]:
# 7日間で順位が下落した銘柄
movers_down = screener.rank_changes(
    metric="composite_score", days=7, direction="down", min_change=10, limit=20
)
print(f"順位下落銘柄: {len(movers_down)}件")
if movers_down.empty:
    print("  → データが1日分しかないか、条件に合う銘柄がありません")
else:
    display(movers_down.head(10))

順位下落銘柄: 0件
  → データが1日分しかないか、条件に合う銘柄がありません


In [43]:
# 異なるメトリクスでの分析
# 注: 複数日分のデータが必要

# HL比率の順位変動
hl_movers = screener.rank_changes(
    metric="hl_ratio", days=7, direction="up", min_change=20
)
print(f"HL比率急上昇: {len(hl_movers)}件")

# RSPの順位変動
rsp_movers = screener.rank_changes(metric="rsp", days=7, direction="up", min_change=20)
print(f"RSP急上昇: {len(rsp_movers)}件")

if hl_movers.empty and rsp_movers.empty:
    print("\n→ データ蓄積後に再実行してください")

HL比率急上昇: 0件
RSP急上昇: 0件

→ データ蓄積後に再実行してください


---
## 10. 銘柄別スコア履歴

`history()`メソッドで特定銘柄の時系列スコアデータを取得できます。

In [44]:
# トヨタの過去30日間のスコア履歴
history = screener.history("7203", days=30)
print(f"履歴データ: {len(history)}日分")
history.head(10)

履歴データ: 0日分


,Date,Code,composite_score,composite_score_rank,hl_ratio_rank,rsp_rank


In [45]:
# 履歴データの可視化
import plotly.express as px

if not history.empty:
    history_sorted = history.sort_values("Date")
    fig = px.line(
        history_sorted,
        x="Date",
        y="composite_score",
        title="7203: 統合スコア推移（30日間）",
        markers=True,
    )
    fig.show()

---
## 11. 実践ワークフロー: スクリーニングから詳細分析へ

実際の投資分析では、以下のワークフローが効果的です：

1. **Stock Screener** で候補銘柄を絞り込み
2. **Market Reader** で詳細な価格データを取得
3. **Technical Tools** でチャート分析・シグナル検出
4. 投資判断

In [46]:
# ステップ1: 有望銘柄をスクリーニング
screening_config = ScreenerFilter(
    composite_score_min=75.0,
    hl_ratio_min=80.0,
    rsi_max=70.0,  # 過熱感なし
    market_cap_min=50_000_000_000,  # 500億円以上
    limit=10,
)

candidates = screener.filter(screening_config)
print(f"候補銘柄数: {len(candidates)}")
print()

# 上位5銘柄を表示
display_cols = ["Code", "composite_score", "composite_score_rank", "HlRatio"]
if "longName" in candidates.columns:
    display_cols.insert(1, "longName")
if not candidates.empty:
    print(candidates[display_cols].head())

候補銘柄数: 10

    Code longName  composite_score  composite_score_rank     HlRatio
0  52010      ＡＧＣ        85.115227                   742  100.000000
1  40420      東ソー        85.005343                   746   99.025788
2  40450     東亞合成        84.742160                   758  100.000000
3  64580     新晃工業        84.635123                   762   98.076923
5  34020       東レ        84.359767                   777  100.000000


In [47]:
# ステップ2: 上位銘柄のコードを取得し、既存分析結果を確認
if not candidates.empty:
    top_code = candidates.iloc[0]["Code"]
    print(f"詳細分析対象: {top_code}")

    # 既存分析結果を確認
    existing = analyzer_jp.load_existing_analysis(top_code)

    if existing["minervini"]:
        print(
            f"\nMinervini条件通過: {existing['minervini'].get('PassesMinervini', 'N/A')}"
        )

    if existing["relative_strength"]:
        rsp = existing["relative_strength"].get("RelativeStrengthPercentage", "N/A")
        print(f"相対力(RSP): {rsp}")

詳細分析対象: 52010

Minervini条件通過: N/A
相対力(RSP): 11.276343845319543


In [48]:
# ステップ3: 詳細チャート分析
if not candidates.empty:
    top_code = candidates.iloc[0]["Code"]

    # フルチャートを描画
    fig = analyzer_jp.plot_chart(
        top_code,
        show_sma=[5, 25, 75],
        show_bb=True,
        show_rsi=True,
        show_macd=True,
        show_signals=True,
        start="2025-01-01",
    )
    fig.update_layout(title=f"{top_code} 詳細テクニカル分析")
    fig.show()

In [49]:
# ステップ4: 最近のシグナルを確認
if not candidates.empty:
    top_code = candidates.iloc[0]["Code"]

    signals = analyzer_jp.detect_crosses(
        top_code,
        patterns=[(5, 25), (25, 75)],
        start="2025-07-01",  # 直近半年
    )

    print(f"\n{top_code} 直近のシグナル:")
    for sig in signals[-5:]:
        signal_ja = (
            "ゴールデンクロス" if sig.signal_type == "golden_cross" else "デッドクロス"
        )
        print(
            f"  {sig.date.strftime('%Y-%m-%d')}: {signal_ja} ({sig.short_period}/{sig.long_period}日) @ {sig.price:.2f}円"
        )


52010 直近のシグナル:
  2025-11-05: ゴールデンクロス (5/25日) @ 5126.00円
  2025-12-09: デッドクロス (5/25日) @ 5180.00円
  2025-12-25: ゴールデンクロス (5/25日) @ 5286.00円
  2025-12-30: デッドクロス (5/25日) @ 5193.00円
  2026-01-07: ゴールデンクロス (5/25日) @ 5378.00円


In [50]:
# ステップ5: スコア推移を確認
if not candidates.empty:
    top_code = candidates.iloc[0]["Code"]

    history = screener.history(top_code, days=14)

    if not history.empty:
        history_sorted = history.sort_values("Date")
        print(f"\n{top_code} 直近2週間のスコア推移:")
        print(
            history_sorted[
                ["Date", "composite_score", "composite_score_rank"]
            ].to_string(index=False)
        )


52010 直近2週間のスコア推移:
      Date  composite_score  composite_score_rank
2026-02-03        85.115227                   742


---
## 12. 複数銘柄の比較分析

スクリーニングで絞り込んだ複数銘柄を比較分析します。

In [51]:
# 複数銘柄のリターン比較チャート
import plotly.graph_objects as go
from market_reader import DataReader

# DataReaderの初期化（セルを独立して実行可能にするため）
if "reader" not in dir():
    reader = DataReader()

# 比較する銘柄を指定（スクリーニング結果があればそれを使用、なければ固定銘柄）
if "candidates" in dir() and not candidates.empty and len(candidates) >= 3:
    codes = candidates["Code"].tolist()[:3]
    print("スクリーニング結果から上位3銘柄を使用")
else:
    codes = ["7203", "9984", "6758"]  # トヨタ、ソフトバンクG、ソニー
    print("デフォルト銘柄を使用: トヨタ、ソフトバンクG、ソニー")

print(f"比較対象: {codes}")

# Market Readerで価格データを取得
df_compare = reader.get_prices(codes, start="2025-01-01")
print(f"取得データ: {len(df_compare)} rows")

if not df_compare.empty:
    fig = go.Figure()

    for code in codes:
        # DataReaderは4桁コードに正規化するため、検索用に4桁に変換
        code_4digit = code[:4] if len(code) == 5 else code
        try:
            code_data = df_compare.xs(code_4digit, level="Code")
            if not code_data.empty:
                first_close = code_data["Close"].iloc[0]
                returns = (code_data["Close"] / first_close - 1) * 100

                fig.add_trace(
                    go.Scatter(
                        x=code_data.index, y=returns, name=code_4digit, mode="lines"
                    )
                )
                print(f"  {code_4digit}: OK ({len(code_data)} rows)")
        except KeyError:
            print(f"  {code_4digit}: データなし")
            continue

    if len(fig.data) > 0:
        fig.update_layout(
            title="銘柄別 リターン比較（2025年年初来）",
            xaxis_title="日付",
            yaxis_title="リターン (%)",
            hovermode="x unified",
        )
        fig.show()
    else:
        print("表示できるデータがありません")
else:
    print("価格データが取得できませんでした")

スクリーニング結果から上位3銘柄を使用
比較対象: ['52010', '40420', '40450']
取得データ: 792 rows
  5201: OK (264 rows)
  4042: OK (264 rows)
  4045: OK (264 rows)


---
## 13. Tips & ベストプラクティス

### DataReader
- 大量データ取得時は`columns`を絞ってメモリ節約
- 複数銘柄のMultiIndexは`xs()`や`loc`で効率的にアクセス
- `strict=True`で本番環境、`strict=False`で探索的分析

### TechnicalAnalyzer
- `get_prices()`は内部キャッシュするため、同一銘柄は再取得不要
- チャートは`fig.write_html("chart.html")`で保存可能
- 米国株は`period="1y"`などyfinance形式のパラメータも使用可能

### StockScreener
- `ScreenerFilter`で定型フィルタを定義し再利用
- `rank_changes()`で急騰・急落銘柄を早期発見
- `history()`でスコアの安定性を確認

### パフォーマンス
- データベースにはインデックスが設定済み（高速クエリ）
- Plotlyチャートは銘柄数が多いと重くなるため個別表示推奨

In [52]:
# チャートのHTML保存（参考）
# fig.write_html("output/7203_analysis.html")
print("fig.write_html('output/7203_analysis.html') でチャートを保存できます")

fig.write_html('output/7203_analysis.html') でチャートを保存できます


---
## 14. まとめ

このノートブックでは、3つのパッケージを使った株式分析ワークフローを説明しました。

### 概要

1. **Market Reader** (`DataReader`)
   - J-Quantsデータベースからの効率的なデータ取得
   - 単一/複数銘柄、カラム選択、日付範囲指定

2. **Technical Tools** (`TechnicalAnalyzer`)
   - テクニカル指標の計算（SMA, EMA, RSI, MACD, BB）
   - クロスシグナルの自動検出
   - インタラクティブチャートの生成

3. **Stock Screener** (`StockScreener`, `ScreenerFilter`)
   - テクニカル・ファンダメンタル複合スクリーニング
   - ランク変動分析による急騰銘柄発見
   - 銘柄履歴によるトレンド確認

### 次のステップ
- 独自のスクリーニング条件を定義してみる
- 定期的なスクリーニング結果を追跡する仕組みを構築
- 他の分析指標（Minervini条件など）との組み合わせ